In [3]:
pip install numpy --upgrade

  Using cached numpy-1.23.1-cp38-cp38-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\roy08\\anaconda3\\Lib\\site-packages\\~-mpy\\core\\multiarray.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns

train=pd.read_csv('train.csv')
test= pd.read_csv('test.csv')

ImportError: this version of pandas is incompatible with numpy < 1.16.5
your numpy version is 1.14.6.
Please upgrade numpy to >= 1.16.5 to use this pandas version

In [ ]:
train

In [ ]:
train['measurement_avg']=train[[f'measurement_{i}' for i in range(3,17)]].mean(axis=1)
test['measurement_avg']=test[[f'measurement_{i}' for i in range(3,17)]].mean(axis=1)


In [ ]:
#data처리-1
# 안 쓰는 변수 제거 measurement_3~16
print('Before',train.shape,test.shape)

train=train.drop([ 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16'],axis=1)

test=test.drop([ 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13',
       'measurement_14', 'measurement_15', 'measurement_16'],axis=1)


combine = [train,test]

print('After',train.shape,test.shape)

In [ ]:
train

In [ ]:
np.unique(train['attribute_3'])

In [ ]:
train.isnull().sum()

In [ ]:

g= sns.FacetGrid(train,col='failure')
g.map(plt.hist,'measurement_avg',bins=40)

In [ ]:
# product_code, attribute_0, attribute_1 를 숫자로 바꿈

# product_code 을숫자로 바꿈
# for dataset in [train,test]:
#     dataset['product_code']=dataset['product_code'].map({'A':1,'B':2,'C':3,'D':4,'E':5}).astype(int)


for dataset in [train,test]:
    dataset['attribute_0']=dataset['attribute_0'].map({'material_7':2,'material_5':1}).astype(int)

for dataset in [train,test]:
    dataset['attribute_1']=dataset['attribute_1'].map({'material_8':3,'material_5':1,'material_6':2}).astype(int)


In [ ]:
train

In [ ]:
train['measurement_avg'].corr(train['failure'])

In [ ]:
train['measurement_2'].corr(train['failure'])

In [ ]:
train.corr()

In [ ]:
#https://dining-developer.tistory.com/19

# KNN 으로 null 채우기 - train
!pip install impyute
from impyute.imputation.cs import fast_knn
np_imputed=fast_knn(train.values, k=5)# KNN 학습 
train_imputed = pd.DataFrame(np_imputed)
np_imputed=fast_knn(test.values, k=5)# KNN 학습 
test_imputed = pd.DataFrame(np_imputed)


In [ ]:
test.isnull().sum()

In [ ]:
train_imputed.isnull().sum()

In [ ]:
for a in range(13):
    b=['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_17', 'failure', 'measurement_avg']
    train_imputed.rename(columns={a:b[a]},inplace=True)
for a in range(12):
    b= ['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_17','measurement_avg']
    test_imputed.rename(columns={a:b[a]},inplace=True)


In [ ]:
train_imputed.head()
test_imputed.head()

In [ ]:
# failure, measurement_avg order change
train_imputed = train_imputed[['id', 'product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_17', 'measurement_avg','failure' ]]
train_imputed

In [ ]:
# normalization
from sklearn import preprocessing
train_normalized = preprocessing.normalize(train_imputed, axis=0)
test_normalized = preprocessing.normalize(test_imputed, axis=0)

In [ ]:
train_normalized.shape, test_normalized.shape

In [ ]:
# X_train=train.drop(['failure'],axis=1)
# Y_train=train['failure']
# X_test=test.copy()

# X_train.shape,Y_train.shape,X_test.shape